<a href="https://colab.research.google.com/github/mapsguy/programming-gemini/blob/main/JSON_EnumConstraints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#step 1: install/upgrade the latest genai SDK
%pip install google-genai --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.0/200.0 kB 13.0 MB/s eta 0:00:00


In [3]:
#import the genai library
from google import genai

In [4]:
#step 2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_API_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [5]:
model_name = "models/gemini-2.5-flash-preview-05-20"

In [6]:
#step 3: define the JSON type returned by the model

from pydantic import BaseModel
#defining the schema for the individual items
class Item(BaseModel):
    name: str
    approx_price_per_unit: float
    quantity: int

#defining the schema for the list of items
class Response(BaseModel):
    items: list[Item]

In [7]:
#step 4: call the model

#ensure types import
from google.genai import types

#set response_mime_type to 'application/json'
#provide response_schema: defines JSON output structure

response = client.models.generate_content(
    model = model_name,
    contents = "Extract the name and quantity of all items mentioned: 3 apples, 10 bananas, 1 apple where each costs around 2, 0.5 and 1 dollar each.",
    config={
        "response_mime_type":"application/json",
        "response_schema":list[Response],
    }
)

In [8]:
#Step 5: Parsing the JSON Response
import json
output_json = json.loads(response.text)
print(json.dumps(output_json, indent=2))

[
  {
    "items": [
      {
        "name": "apples",
        "approx_price_per_unit": 2,
        "quantity": 3
      },
      {
        "name": "bananas",
        "approx_price_per_unit": 0.5,
        "quantity": 10
      },
      {
        "name": "apple",
        "approx_price_per_unit": 1,
        "quantity": 1
      }
    ]
  }
]


In [41]:
#step 6: enum constrained output: forcing specific choices

#ensure enum import
import enum

#define enum schema
class Sentiment(enum.Enum):
  POSITIVE = "Positive"
  NEGATIVE = "Negative"
  NEUTRAL = "Neutral"
  UNKNOWN = "Unknown"


response = client.models.generate_content(
    model = model_name,
    contents = "The new movie was awesome!",
    config={
       "response_mime_type":"text/x.enum",
        "response_schema" : Sentiment,
        #alternative way to send enum constraint below
        #"response_schema" : {
        #    "type": "STRING",
        #    "enum": ["Positive", "Negative", "Neutral", "Unknown"],
        #}
    },
)

print(response.text)


Positive
